In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 2.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import pandas as pd
import sklearn
import numpy as np
from  catboost import CatBoostRegressor

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/data/

train_peptides = pd.read_csv("train_peptides.csv")
train_proteins = pd.read_csv("train_proteins.csv")
train_clinical = pd.read_csv("train_clinical_data.csv")

/content/drive/My Drive/data


In [ ]:
print(train_clinical.shape)
train_clinical.head(10)

(2615, 8)


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On
5,55_18,55,18,7.0,13.0,38.0,0.0,On
6,55_24,55,24,16.0,9.0,49.0,0.0,On
7,55_30,55,30,14.0,13.0,49.0,0.0,On
8,55_36,55,36,17.0,18.0,51.0,0.0,On
9,55_42,55,42,12.0,20.0,41.0,0.0,On


In [ ]:
print(train_peptides.shape)
train_peptides.head(10)

In [ ]:
print(train_proteins.shape)
train_proteins.head(10)

In [ ]:
merged_proteins_peptides = pd.merge(train_proteins, train_peptides, on = ['visit_id', 'visit_month', 'patient_id', 'UniProt'])
merged_df = pd.merge(merged_proteins_peptides, train_clinical, on = ['visit_id', 'visit_month', 'patient_id'])

pivoted_df = merged_df.pivot(index = 'visit_id', columns = ['Peptide'], values = 'PeptideAbundance')

final_df = pd.merge(train_clinical, pivoted_df, on = 'visit_id', how = 'right').set_index('visit_id')
final_df.insert(6, 'visit_month', final_df.pop('visit_month'))
final_df = final_df.drop('patient_id', axis = 1)
#final_df = final_df.dropna()

#final_df.loc[:, 'AADDTWEPFASGK':] = final_df.loc[:, 'AADDTWEPFASGK':].fillna(0)
#final_df.loc[:, 'updrs_1':'updrs_4'] = final_df.loc[:, 'updrs_1':'updrs_4'].fillna(0)

final_df = final_df.drop('upd23b_clinical_state_on_medication', axis = 1)

final_df

#for col in final_df.columns:
  #print(f'NaN values in {col}: {final_df[col].isna().sum()}')

,updrs_1,updrs_2,updrs_3,updrs_4,visit_month,AADDTWEPFASGK,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,AAFTEC(UniMod_4)C(UniMod_4)QAADK,AANEVSSADVK,AATGEC(UniMod_4)TATVGKR,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
visit_id,,,,,,,,,,,,,,,,,,,,,
10053_0,3.0,0.0,13.0,0.0,0,6580710.0,31204.4,7735070.0,NaN,NaN,...,202274.0,NaN,4401830.0,77482.6,583075.0,76705.7,104260.0,530223.0,NaN,7207.30
10053_12,4.0,2.0,8.0,0.0,12,6333510.0,52277.6,5394390.0,NaN,NaN,...,201009.0,NaN,5001750.0,36745.3,355643.0,92078.1,123254.0,453883.0,49281.9,25332.80
10053_18,2.0,2.0,0.0,0.0,18,7129640.0,61522.0,7011920.0,35984.7,17188.00,...,220728.0,NaN,5424380.0,39016.0,496021.0,63203.6,128336.0,447505.0,52389.1,21235.70
10138_12,3.0,6.0,31.0,0.0,12,7404780.0,46107.2,10610900.0,NaN,20910.20,...,188362.0,9433.71,3900280.0,48210.3,328482.0,89822.1,129964.0,552232.0,65657.8,9876.98
10138_24,4.0,7.0,19.0,10.0,24,13788300.0,56910.3,6906160.0,13785.5,11004.20,...,206187.0,6365.15,3521800.0,69984.6,496737.0,80919.3,111799.0,NaN,56977.6,4903.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8699_24,11.0,10.0,13.0,2.0,24,6312970.0,44462.7,12455000.0,11051.3,1163.18,...,289888.0,8615.27,8770410.0,33599.1,926094.0,118897.0,133682.0,571879.0,80268.3,54889.70
942_12,5.0,2.0,25.0,0.0,12,11289900.0,46111.7,11297300.0,NaN,13894.10,...,173259.0,4767.63,374307.0,35767.3,250397.0,65966.9,77976.8,486239.0,45032.7,NaN
942_24,2.0,3.0,23.0,NaN,24,10161900.0,32145.0,12388000.0,25869.2,17341.80,...,185428.0,5554.53,NaN,64049.8,479473.0,68505.7,74483.1,561398.0,52916.4,21847.60


In [ ]:
X = final_df.drop(["updrs_1", "updrs_2", "updrs_3", "updrs_4"], axis = 1)

Y_1 = final_df.loc[:, 'updrs_1']

Y_2 = final_df.loc[:, 'updrs_2']

Y_3 = final_df.loc[:, 'updrs_3']

Y_4 = final_df.loc[:, 'updrs_4']

In [ ]:
X_1 = X.loc[np.logical_not(Y_1.isna()), :]
X_2 = X.loc[np.logical_not(Y_2.isna()), :]
X_3 = X.loc[np.logical_not(Y_3.isna()), :]
X_4 = X.loc[np.logical_not(Y_4.isna()), :]

Y_1.dropna(inplace = True)
Y_2.dropna(inplace = True)
Y_3.dropna(inplace = True)
Y_4.dropna(inplace = True)

In [ ]:
model = CatBoostRegressor(task_type='GPU', iterations=1000, depth=8)

In [ ]:
cv = sklearn.model_selection.RepeatedKFold(n_splits=5, n_repeats=1, random_state=1)

scores_1 = sklearn.model_selection.cross_validate(model, X_1, Y_1, scoring=['explained_variance','neg_mean_squared_error'], cv=cv, n_jobs=None)
scores_2 = sklearn.model_selection.cross_validate(model, X_2, Y_2, scoring=['explained_variance','neg_mean_squared_error'], cv=cv, n_jobs=None)
scores_3 = sklearn.model_selection.cross_validate(model, X_3, Y_3, scoring=['explained_variance','neg_mean_squared_error'], cv=cv, n_jobs=None)
scores_4 = sklearn.model_selection.cross_validate(model, X_4, Y_4, scoring=['explained_variance','neg_mean_squared_error'], cv=cv, n_jobs=None)

Выходные данные были обрезаны до нескольких последних строк (5000).
4:	learn: 3.0389297	total: 899ms	remaining: 2m 58s
5:	learn: 3.0227602	total: 1.07s	remaining: 2m 57s
6:	learn: 2.9986981	total: 1.25s	remaining: 2m 57s
7:	learn: 2.9764342	total: 1.41s	remaining: 2m 55s
8:	learn: 2.9556395	total: 1.6s	remaining: 2m 56s
9:	learn: 2.9331330	total: 1.77s	remaining: 2m 55s
10:	learn: 2.9152926	total: 1.95s	remaining: 2m 55s
11:	learn: 2.8974143	total: 2.12s	remaining: 2m 54s
12:	learn: 2.8800714	total: 2.3s	remaining: 2m 54s
13:	learn: 2.8633375	total: 2.47s	remaining: 2m 54s
14:	learn: 2.8473793	total: 2.64s	remaining: 2m 53s
15:	learn: 2.8344994	total: 2.81s	remaining: 2m 52s
16:	learn: 2.8227410	total: 2.98s	remaining: 2m 52s
17:	learn: 2.7994393	total: 3.15s	remaining: 2m 51s
18:	learn: 2.7745621	total: 3.33s	remaining: 2m 51s
19:	learn: 2.7538697	total: 3.5s	remaining: 2m 51s
20:	learn: 2.7333402	total: 3.68s	remaining: 2m 51s
21:	learn: 2.7151897	total: 3.85s	remaining: 2m 51s
22:	l

In [ ]:
for metrics in scores_1:
  scores_1[metrics] = abs(scores_1[metrics])
  print('{}: {}'.format(metrics, scores_1[metrics].mean()))

print(' ')

for metrics in scores_2:
  print('{}: {}'.format(metrics, scores_2[metrics].mean()))

print(' ')

for metrics in scores_3:
  print('{}: {}'.format(metrics, scores_3[metrics].mean()))

print(' ')

for metrics in scores_4:
  print('{}: {}'.format(metrics, scores_4[metrics].mean()))

print(' ')

NameError: ignored